In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
# read in the data
df = pd.read_csv('matchups 22-23.csv')

# drop unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# rename the Schl column to School
df = df.rename(columns={'Schl': 'team'})

# reanme the Opp to team_2
df = df.rename(columns={'Opp': 'team_2'})

# rename MOV to GPTS
df = df.rename(columns={'MOV': 'GPTS'})

# drop the Year, Date, Rk., Opp, PTS, OPP columns
df = df.drop(columns=['Year', 'Date', 'Rk.', 'Rk..1', 'PTS', 'OPP'])

# read the kaggle Data
kaggle = pd.read_csv('kaggle.csv')

# map the kaggle id to team name
kaggle_dict = {}
for index, row in kaggle.iterrows():
    kaggle_dict[row['team name ']] = row['TeamID']

# create a new column in the dataframe with the kaggle id using the dictionary and the team name
df['kaggle_id'] = df['team'].map(kaggle_dict)
df['kaggle_id_2'] = df['team_2'].map(kaggle_dict)

# if kaggle_id_2 is less than kaggle_id, swap the two columns and swap Schl and Opp as well as GPTS and GOPTS, and make GPTSDiff negative
df.loc[df['kaggle_id_2'] < df['kaggle_id'], 'GPTS'] = -df.loc[df['kaggle_id_2'] < df['kaggle_id'], 'GPTS']
df.loc[df['kaggle_id_2'] < df['kaggle_id'], ['kaggle_id', 'kaggle_id_2', 'team', 'team_2']] = df.loc[df['kaggle_id_2'] < df['kaggle_id'], ['kaggle_id_2', 'kaggle_id', 'team_2', 'team']].values

# remove duplicate matchups
df = df.drop_duplicates(subset=['kaggle_id', 'kaggle_id_2'], keep='first')

# create a new column outcome with 1 if the team won and 0 if the team lost
df['outcome'] = np.where(df['GPTS'] > 0, 1, 0)

df = df.drop(columns=['GPTS'])

# save to matchups.csv
df.to_csv('matchups.csv', index=False)